Recreating SIR+Q Model Presented by Rizi A, Keating L, Gleeson J et al in their study 'The Unreasonable Effectiveness of Contact Tracing on Networks with' Cliques' in Epydemic

Link to paper: https://arxiv.org/abs/2304.10405v2

Link to repo: https://github.com/k-rizi/Contact-Tracing-On-Clique-Networks


In [ ]:
from epyc import JSONLabNotebook, ParallelLab
import epyc
import networkx as nx
import random, itertools
import epydemic
import numpy as np
from epydemic import SIR, Monitor, ProcessSequence, rng, ERNetwork, SEIR, SIR_FixedRecovery, NetworkGenerator
import matplotlib.pyplot as plt
from parameters import *

In [ ]:
create_data_output_directory()

## Build a subclass of 

In [ ]:
class RCNetwork(NetworkGenerator):
    # Network parameters
    N = 'N'
    KMEAN = 'kmean'
    CLIQUE_SIZE = 'cSize'

    def __init__(self, params=None, limit=None):
        super(RCNetwork, self).__init__(params, limit)

    def topology(self):
        return 'RegularClique'

    def _generate(self, params):
        N, K, c = params[self.N], params[self.KMEAN], params[self.CLIQUE_SIZE]

        if K % (c - 1) != 0:
            raise Exception('This configuration is not possible')
        else:
            k = int(K / (c - 1))
            m = int(N * k / c)
            nodes = [(u, _k) for u in range(N) for _k in range(k)]
            random.shuffle(nodes)
            cliques = [(u, _k) for u in range(m) for _k in range(c)]
            l = list(zip(cliques, nodes))
            my_list = [j[1][0] for j in l]
            node_collections = [my_list[i:i + c] for i in range(0, len(my_list), c)]
            g = nx.Graph()
            for col in node_collections:
                g.add_edges_from(itertools.combinations(col, 2))
            g.remove_edges_from(nx.selfloop_edges(g))
        return g

In [27]:
class SIRQ(SIR):
    P_DETECT = 'pDetect'

    QUARANTINE = 'epydemic.sir.Q'
    
    INFECTION_TIME = 'infectionTime'  

    def __init__(self):
        super().__init__()

    def build(self, params):
        pI = params[self.P_INFECT]
        pD = params[self.P_DETECT]
        pR = params[self.P_REMOVE]

        self.addCompartment(self.SUSCEPTIBLE, 1)
        self.addCompartment(self.INFECTED, 0)
        self.addCompartment(self.REMOVED, 0.0)
        self.addCompartment(self.QUARANTINE, 0.0)

        self.trackNodesInCompartment(self.INFECTED)
        self.trackEdgesBetweenCompartments(self.SUSCEPTIBLE, self.INFECTED, name=self.SI)

        # Define the Quarantine event
        self.addEventPerElement(self.SI, pD, self.quarantine, name=self.QUARANTINE)
        self.addEventPerElement(self.SI, pI, self.infect, name=self.INFECTED)
        self.addEventPerElement(self.INFECTED, pR, self.remove, name=self.REMOVED)
        

    def atEquilibrium(self, t):
        return len(self.compartment(self.INFECTED)) == 0

    def quarantine(self, t ,e):
        (n, _ ) = e
        self.changeCompartment(n, self.QUARANTINE)

    def initialCompartments(self):
        g = self.network()
        ns = set(g.nodes())
        N = len(ns)

        n = rng.integers(N)
        self.changeCompartment(n, self.INFECTED)
        g.nodes[n][self.INFECTION_TIME] = 0.0

        # mark all other nodes as susceptible
        ns.remove(n)
        for n in ns:
            self.changeInitialCompartment(n, self.SUSCEPTIBLE)

In [ ]:
def ex_1_sirq(lab):
    lab[SIRQ.P_INFECT] = p_infects
    lab[SIRQ.P_DETECT] = p_detect
    lab[SIRQ.P_REMOVE] = p_remove

    # set the topology for the generated network
    lab[RCNetwork.N] = n_RC
    lab[RCNetwork.KMEAN] = k_mean_RC
    lab[RCNetwork.CLIQUE_SIZE] = c_sizes_RC

    lab['ens'] = range(ens)

    # create the model, network generator, and experiment
    p = SIRQ()
    g = RCNetwork()
    e = epydemic.SynchronousDynamics(p, g)

    # run the experiment
    lab.runExperiment(e)

In [ ]:
# Partition the controlled variables into quarters.
pis = np.split(p_infects, 2)
lab = ParallelLab(JSONLabNotebook(get_out_path("ex_1_sirq"), create=True), cores)

In [ ]:
# To avoid memory issues, run experiment for a quarter of the d_detect range, thus reducing total investigation memory by a factor  of 4
for i, pi in enumerate(pis):
    p_infects = pi
    lab.createWith(get_out_path("ex_" + str(i)), ex_1_sirq)